In [1]:
import geopandas as gpd
import pandas as pd 
import json
from area import area

from geopandas.geoseries import *

file_path_of_forests = 'source/waldflaechen.geojson'
forests = gpd.read_file(file_path_of_forests)
forests.insert(0, 'New_ID', range(0, 0 + len(forests)))

forests["area_text"] = (forests["area_text"].replace(',', '.', regex=True).astype(float))

isBerlinForest = forests['land'] == 'Berlin'
berlinForests = forests[isBerlinForest]
# print(len(forests)) # 11522
# print(len(berlinForests)) # 6258

# berlinForests['area_text'].sum() # 282036587.22155315
# berlinForests.to_file('berlinForests.geojson', driver='GeoJSON')

# # 433585749.9072622 => 43358.5749907 hectar Berlin and Brandenburg
# # forests['area_text'].sum()
# # 282036587.22155315 => 28203.658722155315 hectar in Berlin
# # forests[(forests.land == 'Berlin')]['area_text'].sum()
# # forests[(forests.land == 'Brandenburg')]['area_text'].sum() #151549162.68570915

berlinForest = berlinForests.unary_union
berlinForest.crs = "EPSG:5678"
frame = gpd.GeoSeries([berlinForest])
test = gpd.GeoSeries([berlinForest]).to_json()

y = json.loads(test)
y = y['features'][0]['geometry']
waldflaeche = area(y) 
# print(waldflaeche) # 162857690.6703506 in m^2

In [2]:
file_path_of_open_spaces = 'source/gruenflaechen.geojson'
open_spaces = gpd.read_file(file_path_of_open_spaces)

gruenflaeche = open_spaces['KATASTERFL'].sum()
# print(gruenflaeche) # 61475596.6
# print(len(open_spaces)) # 2525
gesamt = waldflaeche + gruenflaeche
print(gesamt/10000)
# 22433.32872703506

22433.32872703506


In [6]:
data = pd.read_csv("source/gruenflaechen.csv", sep=";").replace('-', '0', regex=True)
data["Größe in m² (Kataster)"] = pd.to_numeric(data["Größe in m² (Kataster)"])
print(len(data)) # 2525

data = data.loc[data['Größe in m² (Kataster)'] > 0.0]
print(data["Größe in m² (Kataster)"].sum() / 10000) # 6147.55966

print(len(data)) # 2518

2525
6147.55966
2518


In [8]:
data = gpd.read_file('source/gruenflaechen.geojson')
print(len(data)) # 2525
data = data.loc[data['KATASTERFL'] > 0]
data.count()

# # KENNZEICH     2518
# # BEZIRKNAME    2518
# # ORTSTLNAME    2514
# # OBJARTNAME    2518
# # NAMENR        2518
# # NAMEZUSATZ    1368
# # BAUJAHR        292
# # SANIERJAHR      98
# # KATASTERFL    2518
# # WIDMUNG       2400
# # PLANNR        2517
# # PLANNAME      2516
# # ANRECH_GF     2507
# # geometry      2518

# data['KATASTERFL'].sum() #61475596.6

2525


KENNZEICH     2518
BEZIRKNAME    2518
ORTSTLNAME    2514
OBJARTNAME    2518
NAMENR        2518
NAMEZUSATZ    1368
BAUJAHR        292
SANIERJAHR      98
KATASTERFL    2518
WIDMUNG       2400
PLANNR        2517
PLANNAME      2516
ANRECH_GF     2507
geometry      2518
dtype: int64